In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df3_WS2T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment3_WS2_has_duration.csv")

In [3]:
df3_WS2T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1161800|550469|842233,Send for Assessment,Send for Assessment,25/11/2019 13:19:59,GUEST,Student,5,N,1. App to Assessment,Part 1 - Application ID Workflow,...,MCW,HDCW,Full Degree,AF,AF,NaN,NaN,N,0 days 1 hours 5 minutes 59 seconds,3959.0


In [4]:
numOfWorkflowProcesses = df3_WS2T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df3_WS2T[df3_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df3_WS2T[df3_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df3_WS2T[df3_WS2T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Assessment Outcome - Successful','Assessment Outcome - Not Successful']) ]
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df3_WS2T[df3_WS2T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [11]:
s3_t2=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s3_t2.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step
0,1,46,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Send for Assessment}
1,2,133,16 days 20 hours 26 minutes 30 seconds,9.0 days 22.0 hours 5.0 minutes 1.0 seconds,16.17%,"{Send for Assessment, Assessment Outcome - Hig..."
2,3,145,26 days 16 hours 44 minutes 22 seconds,12.0 days 1.0 hours 33.0 minutes 33.0 seconds,23.22%,{Assessment Outcome - Higher Preference Succes...
3,4,118,47 days 21 hours 26 minutes 8 seconds,24.0 days 23.0 hours 55.0 minutes 7.0 seconds,17.37%,"{OC - Reviewed - Other, Assessment Outcome - H..."
4,5,67,39 days 13 hours 22 minutes 35 seconds,28.0 days 2.0 hours 7.0 minutes 24.0 seconds,14.93%,"{OC - Reviewed - Other, Assessment Outcome - H..."
5,6,34,38 days 14 hours 19 minutes 54 seconds,35.0 days 3.0 hours 9.0 minutes 20.0 seconds,13.24%,{Assessment Outcome - Higher Preference Succes...
6,7,22,61 days 4 hours 58 minutes 31 seconds,22.0 days 2.0 hours 30.0 minutes 23.0 seconds,9.74%,{Assessment Outcome - Higher Preference Succes...
7,8,18,33 days 12 hours 8 minutes 45 seconds,23.0 days 15.0 hours 2.0 minutes 9.0 seconds,10.42%,{Assessment Outcome - Higher Preference Succes...
8,9,17,38 days 18 hours 32 minutes 37 seconds,34.0 days 20.0 hours 2.0 minutes 21.0 seconds,10.46%,{Acacdemic - Assessment Decision - No Decision...
9,10,7,40 days 3 hours 29 minutes 45 seconds,19.0 days 0.0 hours 35.0 minutes 4.0 seconds,8.57%,"{Send for Assessment, AR - Satisfied - previou..."


In [12]:
s3_t2.to_csv('segment3_WS2_table.csv', index=False)



